#GroupBy

- while normal GroupBy takes a collection of elements and produces a grouped collection of elements, here key is dynamically created from elements.

In [ ]:
#GroupBy normal grouping using
import apache_beam as beam
with beam.Pipeline() as pipeline:
    input = (
        pipeline
        | beam.Create(
            ['strawberry', 'raspberry', 'blueberry', 'blackberry', 'banana'])
        | "Grouping" >> beam.GroupBy(lambda s: s[0])
        | "Print" >> beam.Map(print)
  )

('s', ['strawberry'])
('r', ['raspberry'])
('b', ['blueberry', 'blackberry', 'banana'])


#Aggregation

- Grouping is often used in conjunction with aggregation, and the aggregate_field method of the GroupBy transform can be used to accomplish this easily.
- This method takes three parameters: the field (or expression) which to aggregate, the CombineFn (or associative callable) with which to aggregate by, and finally a field name in which to store the result.

In [ ]:
GROCERY_LIST = [
    beam.Row(recipe='pie', fruit='strawberry', quantity=3, unit_price=1.50),
    beam.Row(recipe='pie', fruit='raspberry', quantity=1, unit_price=3.50),
    beam.Row(recipe='pie', fruit='blackberry', quantity=1, unit_price=4.00),
    beam.Row(recipe='pie', fruit='blueberry', quantity=1, unit_price=2.00),
    beam.Row(recipe='muffin', fruit='blueberry', quantity=2, unit_price=2.00),
    beam.Row(recipe='muffin', fruit='banana', quantity=3, unit_price=1.00),
]

In [ ]:
#grouping using key
with beam.Pipeline() as p:
  input = (
      p
      | "Read List" >> beam.Create(GROCERY_LIST)
      | "Grouping" >> beam.GroupBy('recipe')
      | "Print Result" >> beam.Map(print)
  )

('pie', [BeamSchema_85b7e927_0a2e_4283_8b59_bb1f053a7dd5(recipe='pie', fruit='strawberry', quantity=3, unit_price=1.5), BeamSchema_85b7e927_0a2e_4283_8b59_bb1f053a7dd5(recipe='pie', fruit='raspberry', quantity=1, unit_price=3.5), BeamSchema_85b7e927_0a2e_4283_8b59_bb1f053a7dd5(recipe='pie', fruit='blackberry', quantity=1, unit_price=4.0), BeamSchema_85b7e927_0a2e_4283_8b59_bb1f053a7dd5(recipe='pie', fruit='blueberry', quantity=1, unit_price=2.0)])
('muffin', [BeamSchema_85b7e927_0a2e_4283_8b59_bb1f053a7dd5(recipe='muffin', fruit='blueberry', quantity=2, unit_price=2.0), BeamSchema_85b7e927_0a2e_4283_8b59_bb1f053a7dd5(recipe='muffin', fruit='banana', quantity=3, unit_price=1.0)])


In [ ]:
#grouping using aggregate function like sum
with beam.Pipeline() as p:
  input = (
      p
      | "Read List" >> beam.Create(GROCERY_LIST)
      #| "Grouping" >> beam.GroupBy('recipe').aggregate_field('quantity',sum,'total_quantity')
      | "Grouping" >> beam.GroupBy('recipe').aggregate_field('quantity', sum, 'total_quantity').aggregate_field(lambda x: x.quantity * x.unit_price, sum, 'total_price')
      | "Print Result" >> beam.Map(print)
  )
  grouped = (
      p
      | beam.Create(GROCERY_LIST)
      | beam.GroupBy().aggregate_field(
          'unit_price', min, 'min_price').aggregate_field(
                  'unit_price', max, 'max_price')
      | beam.Map(print))

Result(recipe='pie', total_quantity=6, total_price=14.0)
Result(recipe='muffin', total_quantity=5, total_price=7.0)
Result(min_price=1.0, max_price=4.0)


#GroupByKey

- GroupByKey is a Beam transform for processing collections of key/value pairs.
- It’s a parallel reduction operation, analogous to the Shuffle phase of a Map/Shuffle/Reduce-style algorithm.
- The input to GroupByKey is a collection of key/value pairs that represents a multimap, where the collection contains multiple pairs that have the same key, but different values. Given such a collection, you use GroupByKey to collect all of the values associated with each unique key.

In [ ]:
#Groupbykey
records = [("vignesh", [27, "engineer"]),
("neethu", [27, "developer"]),
("farooqui", [26, "data analyst"]),
("sai", [29, "web developer"]),
("tinkle", [28, "fullstack developer"]),
("neethu", 'Employed'),
("sai", 'Unemployed'),
("tinkle", 'Employed'),
("farooqui",'Employed'),
("vignesh", 'Unemployed')]

In [ ]:
with beam.Pipeline() as p:
  input = (
      p
      | "Read List" >> beam.Create(records)
      | "GroupByKey" >> beam.GroupByKey()
      #| "Sorting and format" >> beam.MapTuple(lambda k,vs: (k,sorted(vs)))
      | "Print Result" >> beam.Map(print)
)

('vignesh', [[27, 'engineer'], 'Unemployed'])
('neethu', [[27, 'developer'], 'Employed'])
('farooqui', [[26, 'data analyst'], 'Employed'])
('sai', [[29, 'web developer'], 'Unemployed'])
('tinkle', [[28, 'fullstack developer'], 'Employed'])


#CoGroupByKey

- Aggregates all input elements by their key and allows downstream processing to consume all values associated with the key.
- While GroupByKey performs this operation over a single input collection and thus a single type of input values.
- CoGroupByKey operates over multiple input collections. As a result, the result for each key is a tuple of the values associated with that key in each input collection.

In [ ]:
#CoGroupByKey
with beam.Pipeline() as pipeline:
  student_pairs = pipeline | 'Create icons' >> beam.Create([
      ('vignesh', 'bangalore'),
      ('khaula', 'hyderabad'),
      ('neethu', 'malapur'),
      ('sai', 'chennai'),
  ])

  student_result = pipeline | 'Create durations' >> beam.Create([
      ('vignesh', [15,"FAIL"]),
      ('khaula', [99,"PASS"]),
      ('neethu', [100,"PASS"]),
      ('sai',[ 37,"FAIL"]),
  ])

  join_both = (
      ({'student_pairs': student_pairs, 'student_result': student_result})
      | 'CoGroupByKey' >> beam.CoGroupByKey()
      | 'Print' >> beam.Map(print)
  )

('vignesh', {'student_pairs': ['bangalore'], 'student_result': [[15, 'FAIL']]})
('khaula', {'student_pairs': ['hyderabad'], 'student_result': [[99, 'PASS']]})
('neethu', {'student_pairs': ['malapur'], 'student_result': [[100, 'PASS']]})
('sai', {'student_pairs': ['chennai'], 'student_result': [[37, 'FAIL']]})


In [ ]:
with beam.Pipeline() as pipeline:
  student_pairs = pipeline | 'Create icons' >> beam.Create([
      ('vignesh', 15),
      ('khaula', 99),
      ('neethu', 100),
      ('sai', 37),
  ])

  student_result = pipeline | 'Create durations' >> beam.Create([
      ('vignesh', "FAIL"),
      ('khaula',"PASS"),
      ('neethu',"PASS"),
      ('sai', "FAIL"),
  ])

  plants = (({
      'Marks': student_pairs, 'Result': student_result
  })
  | 'Merge' >> beam.CoGroupByKey()
  | beam.Map(print))

('vignesh', {'Marks': [15], 'Result': ['FAIL']})
('khaula', {'Marks': [99], 'Result': ['PASS']})
('neethu', {'Marks': [100], 'Result': ['PASS']})
('sai', {'Marks': [37], 'Result': ['FAIL']})


#GroupIntoBatches:
Batches the input into desired batch size.

In [ ]:
with beam.Pipeline() as pipeline:
  batches_with_keys = (
      pipeline
      | 'Create produce' >> beam.Create([
          ('spring', '🍓'),
          ('spring', '🥕'),
          ('spring', '🍆'),
          ('spring', '🍅'),
          ('summer', '🥕'),
          ('summer', '🍅'),
          ('summer', '🌽'),
          ('fall', '🥕'),
          ('fall', '🍅'),
          ('winter', '🍆'),
      ])
      | 'Group into batches'  >> beam.GroupIntoBatches(2)
      | beam.Map(print)
  )

('spring', ['🍓', '🥕'])
('spring', ['🍆', '🍅'])
('summer', ['🥕', '🍅'])
('fall', ['🥕', '🍅'])
('summer', ['🌽'])
('winter', ['🍆'])


#Flatten()

- It is a beam transform for PCollection objects that store the same data type.
- Merges multiple PCollection objects into a single logical PCollection.
- Kind of Union operation (takes duplicate)

In [ ]:
with beam.Pipeline() as p:
  input = (p
           | "first set" >> beam.Create({1,2,3,4,5}))
  input_2 = (p
             | "Second set" >> beam.Create({6,7,8,9,1}))
  result = (
      (input, input_2)
      | beam.Flatten()
      | beam.Map(print)
  )

1
6
7
8
9
1
2
3
4
5


#Partition:
- Partition is a Beam transform for PCollection objects that store the same data type. It splits a single PCollection into a fixed number of smaller collections.

- Partition divides the elements of a PCollection according to a partitioning function that you provide.

- The partitioning function contains the logic that determines how to split up the elements of the input PCollection into each resulting partition PCollection.

- The number of partitions must be determined at graph construction time.

- Partition accepts a function that receives the number of partitions, and returns the index of the desired partition for the element. The number of partitions passed must be a positive integer, and it must return an integer in the range 0 to num_partitions-1.

In [ ]:
#Partition using udf
def splitting_data(element,num_of_part):
  return 0 if element%2==0 else 1

number = {11,12,13,44,55,61,77,88,99}
with beam.Pipeline() as p:
  input = (
      p
      | beam.Create(number)
      | "Do Partitions" >> beam.Partition(splitting_data,2)
      #| "Print Partitions" >> beam.Map(print)
  )
input[0] | "Printing first partition" >> beam.io.WriteToText('first_partition')
input[1] | "Printing second partition" >> beam.io.WriteToText('second_partition')

<PCollection[[36]: Printing second partition/Write/WriteImpl/FinalizeWrite.None] at 0x7881e6d8c0a0>